In [1]:
import pickle as pkl;
import byxtal.lattice as gbl;
import byxtal.find_csl_dsc as fcd;
import byxtal.integer_manipulations as iman;
import byxtal.bp_basis as bpb;
import numpy as np;
import math as mt;
import gbpy.util_funcs_create_byxtal as uf

from sympy.matrices import Matrix, eye, zeros;

########## Variable Definitions
## l_Cbpb_Cp:
## Basis of the 2D lattice in the CSLp reference frame for the
## given (hkl) plane.

## l_CRbpb_Cp:
## Basis of the 2D lattice such that the basis vectors have
## length that is greater than rCut and the basis vectors are similar
## in lengths and the angle between them is close to 90.
## Defined in the CSLp reference frame

##############################################################################
l1 = gbl.Lattice('Al'); l_p_po = Matrix(l1.l_p_po); l_po_p = l_p_po.inv();

##############################################################################
##### Load bicrystallographic properties
pkl_name = 'cubic_gb_props.pkl'; jar = open(pkl_name, 'rb');
gb_props = pkl.load(jar); jar.close();
lSig_CSLbpb_CSLp = gb_props['lSig_CSLbpb_CSLp'];
lSig_CSLbpb_Rcut_CSLp = gb_props['lSig_CSLbpb_Rcut_CSLp'];
hkl_sig_inds = gb_props['hkl_sig_inds'];

In [2]:
import os;
import byxtal;
# path = os.path.dirname(byxtal.__file__)+'/tests/';
path = os.path.dirname(byxtal.__file__)+'/data_files/';
csl_pkl = path+l1.pearson+'_Id_csl_common_rotations.pkl';
jar = open(csl_pkl,'rb'); s1=pkl.load(jar); jar.close();

In [3]:
sig_mats = s1['sig_mats'];
csl_mats = s1['csl_mats'];
dsc_mats = s1['dsc_mats'];
csl_bp_props = s1['csl_bp_props'];


lat_par = l1.lat_params['a']; rCut = lat_par*3; A_cut = (rCut+lat_par)**2;

s1_keys = list(hkl_sig_inds.keys());
t_num = []; bpn_list = [];
t1_num = 0; bpn_po1 = Matrix([[1.0],[ 0.0],[ 0.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 0; bpn_po1 = Matrix([[4.0],[-1.0],[-3.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 1; bpn_po1 = Matrix([[0.0],[-1.0],[1.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 1; bpn_po1 = Matrix([[1.0],[-5.0],[5.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 2; bpn_po1 = Matrix([[4.0],[2.0],[1.0]])
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 2; bpn_po1 = Matrix([[13.0],[3.0],[-2.0]])
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 3; bpn_po1 = Matrix([[1.0],[-1.0],[1.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 3; bpn_po1 = Matrix([[2.0],[ 1.0],[2.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 4; bpn_po1 = Matrix([[12.0],[-1.0],[3.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);
t1_num = 4; bpn_po1 = Matrix([[5.0],[-3.0],[1.0]]);
t_num.append(t1_num); bpn_list.append(bpn_po1);


In [4]:
tct1 = 3
t1_num = t_num[tct1]
bpn_po1 = bpn_list[tct1]

sig_id = s1_keys[t1_num]
print("Sig ID: "+ str(t1_num) + " -- "+ sig_id)


Sig ID: 1 -- 5_1


In [5]:

l_csl_p1 = csl_mats[sig_id];
l_csl_po1 = l_p_po*l_csl_p1;


##############################################################################
#### From MBP
sig_id = s1_keys[t1_num];
l_rp_po = fcd.reciprocal_mat(l_csl_po1);
l_po_rp = l_rp_po.inv();
norm_uvec = bpn_po1/bpn_po1.norm();
n1_rp = l_po_rp*norm_uvec;
hkl_ind = iman.int_finder(n1_rp);
hkl1 = (np.array(hkl_ind, dtype='double')).T;
import gbpy.generate_hkl_indices as ghi;
# l_CSLbpb_CSLp = ghi.compute_hkl_bpb(hkl1);
l_Cbpb_Cp = ghi.compute_hkl_bpb(hkl1);
l_CSLbpbSig_CSLp = ghi.gen_Acut_bpb(l_Cbpb_Cp, l_csl_po1, rCut, A_cut);
l_CRbpb_Cp=l_CSLbpbSig_CSLp[0];

##############################################################################

l_bp_po1 = l_csl_po1*l_CRbpb_Cp;

symm_grp_ax = csl_bp_props[sig_id]['symm_grp_ax'];
bp_symm_grp = csl_bp_props[sig_id]['bp_symm_grp'];

l_p2_p1 = Matrix(sig_mats[sig_id]);
gb_ID = uf.get_gb_uID(l_bp_po1, l_p2_p1, l_p_po, bp_symm_grp, symm_grp_ax, sig_id);
print(gb_ID);

Al_S5_1_N1_1_-5_5_N2_-1_1_-7


In [6]:
zCut = 25*l1.lat_params['a'];

threeD_upts, sim_cell2 = uf.create_half_cryst(l_csl_p1, l_CRbpb_Cp, l_p_po, 'upper', zCut)

l_p1_p2 = l_p2_p1.inv(); l_csl_p2 = l_p1_p2*l_csl_p1;
threeD_lpts, sim_cell1 = uf.create_half_cryst(l_csl_p2, l_CRbpb_Cp, l_p_po, 'lower', zCut)

pkl_name = 'gb_attr_'+gb_ID+'.pkl';
jar = open(pkl_name,'wb'); pkl.dump(gb_attr, jar); jar.close();

gb_attr = {};
gb_attr['cell'] = sim_cell1;
gb_attr['upts'] = threeD_upts;
gb_attr['lpts'] = threeD_lpts;

In [7]:
### Points with Y-axis perpendicular to the GB
rot_mat = np.array([[1.0, 0, 0], [0, 0.0, -1.0], [0, 1.0, 0.0]])
threeD_lypts = (np.dot(rot_mat, threeD_upts.transpose())).transpose();
threeD_uypts = (np.dot(rot_mat, threeD_lpts.transpose())).transpose();

sim_cell2 = 0*sim_cell1
sim_cell2[:,0] = sim_cell1[:,0]

sim_cell2[0,1] = sim_cell1[0,2]
sim_cell2[1,1] = sim_cell1[2,2]
sim_cell2[2,1] = sim_cell1[1,2]

sim_cell2[0,2] = sim_cell1[0,1]
sim_cell2[1,2] = sim_cell1[2,1]
sim_cell2[2,2] = sim_cell1[1,1]
sim_cell2[1,3] = sim_cell1[2,3]

gbY_attr = {};
gbY_attr['cell'] = sim_cell2;
gbY_attr['upts'] = threeD_lypts;
gbY_attr['lpts'] = threeD_uypts;

pkl_name = 'gbY_attr_'+gb_ID+'.pkl';
jar = open(pkl_name,'wb'); pkl.dump(gbY_attr, jar); jar.close();

In [15]:
# print(sim_cell2[:,0])
# print(sim_cell2[:,1])
# print(sim_cell2[:,2])
# print(sim_cell2[:,3])

In [9]:
# uf.create_half_cryst(l_csl_p1, l_CRbpb_Cp, l_p_po, 'upper', zCut)
# create_half_cryst(l_csl_p1, l_bp_CSLp, l_p_po, cryst_typ, zCut)
l_bp_po1 = l_csl_po1*l_CRbpb_Cp;
# l_bpb_p1, l2D_bp_po1, twoD_pts = create_twoD_slab(l_bp_po1, l_p_po);

In [10]:
(l_bp_po1/l1.lat_params['a'])

Matrix([
[2.5, -2.5],
[2.0,  3.0],
[1.5,  3.5]])